In [1]:
from datetime import datetime, timezone
from typing import List
from dataclasses import dataclass
from os.path import join
import numpy as np
import pandas as pd
from pynwb import NWBFile, NWBHDF5IO
from pynwb.file import Subject
from pynwb.ecephys import ElectricalSeries, LFP, ElectrodeGroup
import nixio

In [2]:
@dataclass(frozen=True)
class Context:
    subject: int
    session: int
    nix: nixio.File
    nwb: NWBFile

In [3]:
def main() -> Context:
    context = create_context(1, 1)
    write_subject(context)
    write_eeg_electrodes(context)
    return context

In [4]:
def create_context(subject: int, session: int) -> Context:
    nix = _read_nix(subject, session)
    general = nix.sections["General"]
    nwb = NWBFile(
        session_description=_get_task(nix),
        identifier=f"Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM_subject{subject:02}_session{session:02}",
        session_start_time=datetime(2019, 3, 27, 12, 00, tzinfo=timezone.utc),  # TODO: Get this from Johannes
        lab=general.props["Recording location"].values[0],
        institution="Universitätsspital Zürich, 8091 Zurich, Switzerland",  # Broken UTF-8 in file
        related_publications=_get_related_publications(nix)
    )
    return Context(subject, session, nix, nwb)


In [5]:
def _read_nix(subject: int, session: int) -> nixio.File:
    _nix_dir = join("/", "mnt", "c", "Users", "conta", "git", "USZ_NCH", "Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM",
                    "data_nix")
    _file_path = join(_nix_dir, f"Data_Subject_{subject:02}_Session_{session:02}.h5")
    return nixio.File.open(_file_path, nixio.FileMode.ReadOnly)

In [6]:
def _get_task(nix: nixio.File) -> str:
    task = nix.sections["Task"].props
    task_name = task["Task name"].values[0]
    # Broken UTF-8 in file
    task_desc = "The task is a modified Sternberg task in which the encoding of memory items, maintenance, and recall were temporally separated. Each trial starts with a fixation period ([-6, -5] s), followed by the stimulus ([-5, -3] s). The stimulus consists of a set of eight consonants at the center of the screen. The middle four, six, or eight letters are the memory items,which determine the set size for the trial (4, 6, or 8, respectively). The outer positions are filled with “X,” which is never a memory item. After the stimulus, the letters disappear from the screen, and the maintenance interval starts ([-3, 0] s).A fixation square is shown throughout fixation, encoding, and maintenance. After maintenance, a probe letter is presented. The subjects respond with a button press to indicate whether the probe was part of the stimulus.The subjects are instructed to respond as rapidly as possible without making errors. The hand used for the response is counterbalanced across subjects within the clinical constraints. After the response, the probe is turned off, and the subjects receive acoustic feedback regarding whether their response was correct or incorrect. Before initiating the next trial, the subjects were encouraged to blink and relax. The subjects perform 50 trials in one session, which last approximately 10 min. Trials with different set sizes are presented in a random order,with the single exception that a trial with an incorrect response is always followed by a trial with a set size of 4."
    task_url = task["Task URL"].values[0]
    return f"Task Name: {task_name}\nTask Description: {task_desc}\nTask URL: {task_url}"

In [7]:
def _get_related_publications(nix: nixio.File) -> List[str]:
    related_publications = nix.sections["General"].sections["Related publications"].props
    names_and_dois = zip(related_publications["Publication name"].values,
                         related_publications["Publication DOI"].values)
    return [f"{name.strip()} ({doi.strip()})" for (name, doi) in names_and_dois]

In [8]:
def write_subject(context: Context):
    subject = context.nix.sections["Subject"].props
    age = subject["Age"].values[0]
    sex = subject["Sex"].values[0]
    context.nwb.subject = Subject(
        subject_id=f"{context.subject:02}",
        age=f"P{int(age)}Y",
        description=_get_subject_description(context),
        species="Homo sapiens",
        sex=_get_sex(sex),
    )

In [9]:
def _get_sex(raw: str) -> str:
    male = ["male", "m", "männlich", "maennlich", "mannlich", "männchen", "mannchen"]
    female = ["female", "f", "weiblich", "weibchen"]
    sex = raw.lower()
    return "M" if sex in male else "F" if sex in female else "O"

In [10]:
def _get_subject_description(context: Context) -> str:
    subject = context.nix.sections["Subject"].props
    handedness = subject["Handedness"].values[0]
    pathology = subject["Pathology"].values[0]
    depth_electrodes = subject["Depth electrodes"].values[0]
    electrodes_in_soz = subject["Electrodes in seizure onset zone (SOZ)"].values[0]
    return f"Handedness: {handedness}\nPathology: {pathology}\nDepth electrodes: {depth_electrodes}\nElectrodes in seizure onset zone (SOZ): {electrodes_in_soz}"


In [11]:
ELECTRODE_COUNT = 19

In [12]:
def write_eeg_electrodes(context: Context):
    nwb = context.nwb

    device = nwb.create_device(
        name="NicoletOne EEG System", manufacturer="Natus Medical Incorporated"
    )

    # create an electrode group for this group
    electrode_group = nwb.create_electrode_group(
        name=f"eeg",
        description=f"EEG electrodes on scalp",
        device=device,
        location="Scalp",
    )

    electrodes = _get_eeg_electrodes(context)
    electrodes.apply(lambda row: _add_row_to_electrodes(nwb, electrode_group, row), axis=1)

In [13]:
def _add_row_to_electrodes(nwb: NWBFile, electrode_group: ElectrodeGroup, row: pd.Series):
    # got BESA map: +X is anterior, +Y is left, +Z is superior according to <https://eeglab.org/tutorials/ConceptsGuide/coordinateSystem.html>
    # But need NWB: +X is posterior, +Y is inferior, +Z is right according to <https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.NWBFile.add_electrode>
    nwb.add_electrode(
        group=electrode_group,
        location=row["Label"],
        reference="Averaged mastoid channels",
        x=-row["x"],
        y=-row["z"],
        z=-row["y"],
    )

In [14]:
def _get_eeg_electrodes(context: Context) -> pd.DataFrame:
    labels = _get_eeg_electrode_labels(context)
    locations = _get_eeg_electrode_locations(context)
    locations_array = np.ndarray(locations.shape)
    locations.read_direct(locations_array)
    df = pd.DataFrame(locations_array, columns=["x", "y", "z"])
    df.insert(0, "Label", labels)
    return df.reset_index()

In [15]:
def _assert_all_eeg_electrodes_have_same_labels(context: Context):
    data_arrays = _get_session_data(context).groups["Scalp EEG data"].data_arrays
    electrode_labels = [data_array.dimensions[0].labels for data_array in data_arrays]
    assert len(set(electrode_labels)) == 1


In [16]:
def _get_eeg_electrode_labels(context: Context) -> List[str]:
    _assert_all_eeg_electrodes_have_same_labels(context)
    session_data = _get_session_data(context)
    return session_data.groups["Scalp EEG data"].data_arrays["Scalp_EEG_Data_Trial_01"].dimensions[0].labels

In [17]:
def _get_session_data(context: Context) -> nixio.Block:
    return context.nix.blocks[f"Data_Subject_{context.subject:02}_Session_{context.session:02}"]

In [18]:
def _get_eeg_electrode_locations(context: Context) -> nixio.DataArray:
    return _get_session_data(context).groups["Scalp EEG electrode information"].data_arrays[
        "Scalp_Electrode_EEGLAB_BESA_Coordinates"]

In [21]:
if __name__ == "__main__":
    context = main()
    print(context.nwb.electrodes[0])

   location                                              group group_name  \
id                                                                          
2        C3  eeg pynwb.ecephys.ElectrodeGroup at 0x14036512...        eeg   

               x          y          z                  reference  
id                                                                 
2  -3.868125e-15 -56.871691 -63.171281  Averaged mastoid channels  
